In [1]:
# things we need

# NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

#Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

import json
import pickle

curses is not supported on this machine (please install/reinstall curses for an optimal experience)








In [2]:
# import our chat-bot intents file
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [3]:

words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

58 documents
8 classes ['goodbye', 'greeting', 'meanings', 'protect', 'shouldnt', 'spread', 'symptoms', 'update']
59 unique stemmed words ['&', '.', '0', '1', '19', '2', '3', '4', '5', '6', '6.', 'a', 'am', 'and', 'anyon', 'anyth', 'ar', 'ca', 'can', 'cas', 'covid', 'covid-19', 'covid19', 'cya', 'day', 'do', 'doe', 'good', 'goodby', 'hav', 'hello', 'hi', 'how', 'i', 'is', 'lat', 'latest', 'leav', 'mask', 'myself', "n't", 'nee', 'not', 'numb', 'of', 'protect', 'see', 'should', 'spread', 'symptom', 'symptomp', 'the', 'ther', 'to', 'up', 'upd', 'wear', 'what', 'you']


In [4]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [5]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 7999  | total loss: 0.13045 | time: 0.013s
| Adam | epoch: 1000 | loss: 0.13045 - acc: 0.9918 -- iter: 56/58
Training Step: 8000  | total loss: 0.11759 | time: 0.016s
| Adam | epoch: 1000 | loss: 0.11759 - acc: 0.9926 -- iter: 58/58
--
INFO:tensorflow:C:\Users\richa\Project\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [6]:
# save all of our data structures
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )


In [7]:
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

# import our chat-bot intents file
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [8]:
model.load('./model.tflearn')

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\richa\Project\model.tflearn


In [9]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [10]:
p = bow("Should I wear a mask to protect myself?", words)
print (p)

[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 1 1 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 1 0 0]


In [11]:
ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # a random response from the intent
                    return print(random.choice(i['responses']))

            results.pop(0)

In [12]:
classify('Should I wear a mask to protect myself?')

[('protect', 0.99997795)]

In [13]:
response('hi')

Hello! What can i help you with? You can ask me about Covid-19, just write this key-word to explore topics.
 1. What is COVID-19?
 2. Latest Numbers & Cases
 3. Should I wear a mask to protect myself?
 4. How does COVID-19 spread?
 5. Is there anything I should not do?
 6. What are the symptoms of COVID-19?



In [14]:
response('Should I wear a mask to protect myself?')

Only wear a mask if you are ill with COVID-19 symptoms (especially coughing)
 or looking after someone who may have COVID-19. Disposable face mask can only be used once.
 If you are not ill or looking after someone who is ill then you are wasting a mask.
 There is a world-wide shortage of masks, so WHO urges people to use masks wisely.

 WHO advises rational use of medical masks to avoid unnecessary wastage of precious resources and mis-use of masks.

	The most effective ways to protect yourself and others against COVID-19 are to frequently clean your hands,
 cover your cough with the bend of elbow or tissue and maintain
 a distance of at least 1 meter (3 feet) from people who are coughing or sneezing.

Reply '0' for Menu


In [15]:
response('cya')

If you have any other question 


In [16]:
response('hi')

Hello! What can i help you with? You can ask me about Covid-19, just write this key-word to explore topics.
 1. What is COVID-19?
 2. Latest Numbers & Cases
 3. Should I wear a mask to protect myself?
 4. How does COVID-19 spread?
 5. Is there anything I should not do?
 6. What are the symptoms of COVID-19?



In [17]:
response('1')

CO stands for corona, VI for virus, and D for disease. Formerly, this disease was referred to as 2019 novel coronavirus or 2019-nCoV. The COVID-19 virus is a new virus linked to the same family of viruses as Severe Acute Respiratory Syndrome (SARS) and some types of common cold.

Reply '0' for Menu


In [18]:
response('2.')

You can check the latest update here:
 https://covid19.who.int/region/searo/country/id

Reply '0' for Menu


In [19]:
response('how does covid')

People can catch COVID-19 from others who have the virus.

	The disease can spread from person to person through small droplets from the nose or mouth which are spread when a person with COVID-19 coughs or exhales.

	These droplets land on objects and surfaces around the person.

	Other people then catch COVID-19 by touching these objects or surfaces, then touching their eyes, nose or mouth.

	People can also catch COVID-19 if they breathe in droplets from a person with COVID-19 who coughs out or exhales droplets. This is why it is important to stay more than 1 meter (3 feet) away from a person who is sick.

	WHO is assessing ongoing research on the ways COVID-19 is spread and will continue to share updated findings.

Reply '0' for Menu


In [20]:
response('anything i shouldnt do ?')

The following measures ARE NOT effective against COVID-19 and can be harmful:

	Smoking

	Wearing multiple masks

 Taking antibiotics

	In any case, if you have fever, cough and difficulty breathing seek medical care early to reduce the risk of developing a more severe infection and be sure to share your recent travel history with your health care provider.

Reply '0' for Menu


In [21]:
response('0')

Hello! What can i help you with? You can ask me about Covid-19, just write this key-word to explore topics.
 1. What is COVID-19?
 2. Latest Numbers & Cases
 3. Should I wear a mask to protect myself?
 4. How does COVID-19 spread?
 5. Is there anything I should not do?
 6. What are the symptoms of COVID-19?



In [22]:
classify('1.')

[('meanings', 0.9762364)]

In [23]:
response('symptoms')

Hello! What can i help you with? You can ask me about Covid-19, just write this key-word to explore topics.
 1. What is COVID-19?
 2. Latest Numbers & Cases
 3. Should I wear a mask to protect myself?
 4. How does COVID-19 spread?
 5. Is there anything I should not do?
 6. What are the symptoms of COVID-19?



In [24]:
classify('how does covid spread')

[('spread', 0.99993646)]